In [1]:
## needs ~50 gb memory
import pandas as pd 
import numpy as np 

import gc

members = pd.read_csv('../input/members.csv') # more member msno than training examples.  some train msno are not in members.
transactions = pd.read_csv('../input/transactions.csv') 
user_logs = pd.read_csv('../input/user_logs.csv', usecols=['msno','date','total_secs']) # some msno from train are missing.
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/sample_submission_zero.csv')

group = transactions.groupby('msno')
trans_summ_stats = group.agg({'msno' : { 'total_order' : 'count' },
                        'plan_list_price' : {'plan_net_worth' : 'sum'},
                         'actual_amount_paid' : {'mean_payment_each_transaction' : 'mean',
                                                 'total_actual_payment' : 'sum'},
                         'is_cancel' : {'cancel_times' : lambda x : sum(x==1)}})


trans_summ_stats.columns = trans_summ_stats.columns.droplevel(0)
trans_summ_stats.reset_index(inplace=True)

base = pd.merge(left=members, right=trans_summ_stats, how='right', on='msno')
print("1",base.columns)
# add summary stats for the user logs

group = user_logs.groupby('msno')
logs_summ_stats = group.agg({
    'msno': {'total_listens' : 'count'},
    'date': {'last_listen_date' : 'max'}
    })

logs_summ_stats.columns = logs_summ_stats.columns.droplevel(0)
logs_summ_stats.reset_index(inplace=True)

base = base.merge(logs_summ_stats, how='left', on='msno')

# the last time a user played a song - the date the user registered
base['dist_last_listen_registration'] = base['last_listen_date'] - base['registration_init_time'] 
# last listen time - expiration date
base['dist_expiration_last_listen'] = base['expiration_date'] - base['last_listen_date']
base['listens_per_day'] = base['total_listens'] / base['dist_last_listen_registration']

base['gender'].fillna('unknown_gender', inplace=True)
dummies = pd.get_dummies(base['gender'])

base.drop('gender', axis=1, inplace=True)
base = base.join(dummies)
train = train.merge(base, how='left', on='msno')

test = test.merge(base, how='left', on='msno')

train.to_csv('../input/merged-train.csv',index=False)
test.to_csv('../input/merged-test.csv',index=False)

/home/dashiell/anaconda3/lib/python3.6/site-packages/pandas/core/groupby.py:4036: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


1 Index(['msno', 'city', 'bd', 'gender', 'registered_via',
       'registration_init_time', 'expiration_date', 'total_order',
       'plan_net_worth', 'mean_payment_each_transaction',
       'total_actual_payment', 'cancel_times'],
      dtype='object')


In [ ]:
#print(len(train),len(members), len(test))

#train.isnull().sum()

In [2]:
#test

### some msno data is missing..
#transactions[transactions['msno']=='4HykJ6KaHOBgdGIa4mlxU9bxFRc+Co/PGcrhgTftOm0=']  ## has records
#user_logs[user_logs['msno']=='4HykJ6KaHOBgdGIa4mlxU9bxFRc+Co/PGcrhgTftOm0='] ## no records
#members[members['msno']=='4HykJ6KaHOBgdGIa4mlxU9bxFRc+Co/PGcrhgTftOm0=']  ## no records
#train[train['msno']=='4HykJ6KaHOBgdGIa4mlxU9bxFRc+Co/PGcrhgTftOm0=']  ## has records

#train.isnull().sum()

train.head()

,msno,is_churn,city,bd,registered_via,registration_init_time,expiration_date,total_order,plan_net_worth,mean_payment_each_transaction,total_actual_payment,cancel_times,total_listens,last_listen_date,dist_last_listen_registration,dist_expiration_last_listen,listens_per_day,female,male,unknown_gender
0,waLDQMmcOu2jLDaV1ddDkgCrB/jl6sD66Xzs0Vqax1Y=,1,18.0,36.0,9.0,20050406.0,20170907.0,2,149,74.500000,149,0,26.0,20170208.0,119802.0,699.0,0.000217,1,0,0
1,QA7uiXy8vIbUSPOkCf9RwQ3FsT8jVq2OxDr8zqa7bRQ=,1,10.0,38.0,9.0,20050407.0,20170321.0,23,3309,150.347826,3458,2,521.0,20170215.0,119808.0,106.0,0.004349,0,1,0
2,fGwBva6hikQmTJzrbz/2Ezjm5Cth5jZUNvXigKK2AFA=,1,11.0,27.0,9.0,20051016.0,20170203.0,10,1492,149.200000,1492,1,237.0,20170131.0,119115.0,72.0,0.001990,1,0,0
3,mT5V8rEpa+8wuqi6x0DoVd3H5icMKkE9Prt49UlmK+4=,1,13.0,23.0,9.0,20051102.0,20170926.0,2,1788,894.000000,1788,0,735.0,20170221.0,119119.0,705.0,0.006170,1,0,0
4,XaPhtGLk/5UvvOYHcONTwsnH97P4eGECeq+BARGItRw=,1,3.0,27.0,9.0,20051228.0,20170927.0,8,3576,447.000000,3576,0,758.0,20170228.0,119000.0,699.0,0.006370,0,1,0


In [3]:
test.head()

,msno,is_churn,city,bd,registered_via,registration_init_time,expiration_date,total_order,plan_net_worth,mean_payment_each_transaction,total_actual_payment,cancel_times,total_listens,last_listen_date,dist_last_listen_registration,dist_expiration_last_listen,listens_per_day,female,male,unknown_gender
0,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,0,5.0,28.0,3.0,20131223.0,20170822.0,6.0,298.0,149.000000,894.0,1.0,53.0,20170228.0,39005.0,594.0,0.001359,0.0,1.0,0.0
1,f/NmvEzHfhINFEYZTR05prUdr+E+3+oewvweYz9cCQE=,0,13.0,20.0,3.0,20131223.0,20170519.0,5.0,627.0,125.400000,627.0,0.0,74.0,20170227.0,39004.0,292.0,0.001897,0.0,1.0,0.0
2,zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=,0,13.0,18.0,3.0,20131227.0,20170919.0,18.0,2682.0,149.000000,2682.0,0.0,322.0,20170228.0,39001.0,691.0,0.008256,0.0,1.0,0.0
3,8iF/+8HY8lJKFrTc7iR9ZYGCG2Ecrogbc2Vy5YhsfhQ=,0,1.0,0.0,7.0,20140109.0,20160325.0,19.0,2384.0,141.157895,2682.0,3.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0
4,K6fja4+jmoZ5xG6BypqX80Uw/XKpMgrEMdG2edFOxnA=,0,13.0,35.0,7.0,20140125.0,20180218.0,24.0,2529.0,123.166667,2956.0,2.0,256.0,20170227.0,30102.0,9991.0,0.008504,1.0,0.0,0.0
